# Building a Search Engine with BERT and TensorFlow

In this experiment, we will use a pre-trained BERT model checkpoint to build a general-purpose text feature extractor.

These things are sometimes referred to as  [Natural Language Understanding](https://en.wikipedia.org/wiki/Natural-language_understanding) (NLU) modules, because the features they extract are relevant for a wide array of downstream NLP tasks.

One use for these features is in [instance-based learning](https://en.wikipedia.org/wiki/Instance-based_learning), which relies on computing the similarity of the query to the training samples.

We will illustrate this by building a simple Information Retrieval system using the BERT NLU module for feature extraction.


The plan for this experiment is:
1. getting the pre-trained BERT model checkpoint
2. extracting a sub-graph optimized for inference
3. creating a feature extractor with tf.Estimator
4. implementing an Information Retrieval engine
5. accelerating search queries with math
6. building a movie recommendation system

## Step 1: getting the pre-trained model

We start with a pre-trained english BERT-base model checkpoint. 

For configuring and optimizing the graph for inference we will use [bert-as-a-service](https://github.com/hanxiao/bert-as-service) repository, which allows for serving BERT models for remote clients over TCP.

Having a single BERT-server is definitely beneficial in many environments. However,  in this part of the experiment we will focus on creating a local (in-process) feature extractor. This is useful if one wishes to avoid additional latency and potential failure modes introduced by a client-server architecture.

Now, let us download the model and install the package.

In [0]:
!wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
!unzip uncased_L-12_H-768_A-12.zip
!pip install bert-serving-server --no-deps

## Step 2: optimizing the inference graph
Normally, to modify the model graph we would have to do some low-level TensorFlow programming. 

However, thanks to bert-as-a-service, we can configure the inference graph using a simple CLI interface.

There are a couple of parameters in the below snippet too look out for.

For each text sample, BERT-base model encoding layers output a tensor of shape **[sequence_len, encoder_dim],** with one vector per input token. To obtain a fixed representation, we need to apply some sort of pooling.

**POOL_STRAT** parameter defines the pooling strategy applied to the  **POOL_LAYER** encoding layer. The default value **REDUCE_MEAN** averages the vectors for all tokens in a sequence. This strategy works best for most sentence-level tasks, when the model is not fine-tuned. Another option is NONE, in which case no pooling is applied at all. This is useful for word-level tasks such as Named Entity Recognition or POS tagging. For a detailed discussion of other options check out the Han Xiao's [blog post.](https://hanxiao.github.io/2019/01/02/Serving-Google-BERT-in-Production-using-Tensorflow-and-ZeroMQ/)

**SEQ_LEN** affects the maximum length of sequences processed by the model. Smaller values increase the model inference speed almost linearly.

In [0]:
import os
import tensorflow as tf
sesh = tf.InteractiveSession()

from bert_serving.server.graph import optimize_graph
from bert_serving.server.helper import get_args_parser

# input dir
MODEL_DIR = '/content/uncased_L-12_H-768_A-12' #@param {type:"string"}
# output dir
GRAPH_DIR = '/content/graph/' #@param {type:"string"}
# output filename
GRAPH_OUT = 'extractor.pbtxt' #@param {type:"string"}

POOL_STRAT = 'REDUCE_MEAN' #@param ['REDUCE_MEAN', 'REDUCE_MAX', "NONE"]
POOL_LAYER = '-2' #@param {type:"string"}
SEQ_LEN = '256' #@param {type:"string"}

tf.gfile.MkDir(GRAPH_DIR)

parser = get_args_parser()
carg = parser.parse_args(args=['-model_dir', MODEL_DIR,
                               '-graph_tmp_dir', GRAPH_DIR,
                               '-max_seq_len', str(SEQ_LEN),
                               '-pooling_layer', str(POOL_LAYER),
                               '-pooling_strategy', POOL_STRAT])

tmp_name, config = optimize_graph(carg)
graph_fout = os.path.join(GRAPH_DIR, GRAPH_OUT)

tf.gfile.Rename(
    tmp_name,
    graph_fout,
    overwrite=True
)
print("\nSerialized graph to {}".format(graph_fout))

Running the above snippet will put the BERT model graph and weights from  **MODEL_DIR** into a GraphDef object which will be serialized to a pbtxt file at **GRAPH_OUT**. The file will be smaller than the original model because the nodes and variables required for training will be removed. This results in a quite portable solution: for example the english base model only takes 389 MB after exporting.

## Step 3: creating a feature extractor

Now, we will use the serialized graph to build a feature extractor using the tf.Estimator API. We will need to define two things: **input_fn** and **model_fn**

In [0]:
import logging
import numpy as np

from tensorflow.python.estimator.estimator import Estimator
from tensorflow.python.estimator.run_config import RunConfig
from tensorflow.python.estimator.model_fn import EstimatorSpec
from tensorflow.keras.utils import Progbar

from bert_serving.server.bert.tokenization import FullTokenizer
from bert_serving.server.bert.extract_features import convert_lst_to_features


log = logging.getLogger('tensorflow')
log.setLevel(logging.INFO)
log.handlers = []

In [0]:
GRAPH_PATH = "/content/graph/extractor.pbtxt" #@param {type:"string"}
VOCAB_PATH = "/content/uncased_L-12_H-768_A-12/vocab.txt" #@param {type:"string"}

SEQ_LEN = 256 #@param {type:"integer"}

## input_fn

**input_fn** manages getting the data into the model. That includes executing the whole text preprocessing pipeline and preparing a feed_dict for BERT. 

First, each text sample is converted into a tf.Example instance containing the necessary features listed in **INPUT_NAMES**. The bert_tokenizer object contains  the WordPiece vocabulary and performs the text preprocessing. After that the examples are re-grouped by feature name in a **feed_dict**.

In [0]:
INPUT_NAMES = ['input_ids', 'input_mask', 'input_type_ids']
bert_tokenizer = FullTokenizer(VOCAB_PATH)

def build_feed_dict(texts):
    
    text_features = list(convert_lst_to_features(
        texts, SEQ_LEN, SEQ_LEN, 
        bert_tokenizer, log, False, False))

    target_shape = (len(texts), -1)

    feed_dict = {}
    for iname in INPUT_NAMES:
        features_i = np.array([getattr(f, iname) for f in text_features])
        features_i = features_i.reshape(target_shape).astype("int32")
        feed_dict[iname] = features_i

    return feed_dict

tf.Estimators have a fun feature which makes them re-build and re-initialize the whole computational graph at each call to the predict function. 

So, in order to avoid the overhead, to the predict function we will pass a generator, which will yield the features to the model in a never-ending loop. Ha-ha.

In [0]:
def build_input_fn(container):
    
    def gen():
        while True:
          try:
            yield build_feed_dict(container.get())
          except:
            yield build_feed_dict(container.get())

    def input_fn():
        return tf.data.Dataset.from_generator(
            gen,
            output_types={iname: tf.int32 for iname in INPUT_NAMES},
            output_shapes={iname: (None, None) for iname in INPUT_NAMES})
    return input_fn

class DataContainer:
  def __init__(self):
    self._texts = None
  
  def set(self, texts):
    if type(texts) is str:
      texts = [texts]
    self._texts = texts
    
  def get(self):
    return self._texts

## model_fn

model_fn contains the specification of the model. In our case, it is loaded from the pbtxt file we saved in the previous step. 

The features are mapped explicitly to the corresponding input nodes with input_map.

In [0]:
def model_fn(features, mode):
    with tf.gfile.GFile(GRAPH_PATH, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        
    output = tf.import_graph_def(graph_def,
                                 input_map={k + ':0': features[k] for k in INPUT_NAMES},
                                 return_elements=['final_encodes:0'])

    return EstimatorSpec(mode=mode, predictions={'output': output[0]})
  
estimator = Estimator(model_fn=model_fn)

Now we have everything we need to perform inference. Let's do this!

In [0]:
def batch(iterable, n=1):
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]

def build_vectorizer(_estimator, _input_fn_builder, batch_size=128):
  container = DataContainer()
  predict_fn = _estimator.predict(_input_fn_builder(container), yield_single_examples=False)
  
  def vectorize(text, verbose=False):
    x = []
    bar = Progbar(len(text))
    for text_batch in batch(text, batch_size):
      container.set(text_batch)
      x.append(next(predict_fn)['output'])
      if verbose:
        bar.add(len(text_batch))
      
    r = np.vstack(x)
    return r
  
  return vectorize

In [0]:
bert_vectorizer = build_vectorizer(estimator, build_input_fn)

In [0]:
bert_vectorizer(64*['sample text']).shape

Consecutive calls will not rebuild the model.

In [0]:
bert_vectorizer(256*['sample text']).shape

## Step 4: exploring vector space with Projector

A standalone version of BERT feature extractor is available in the [repository](https://github.com/gaphex/bert_experimental).

Now it's time for a demonstration!

Using the vectorizer we will generate embeddings for articles from the Reuters-21578 benchmark corpus.
To visualise and explore the embedding vector space in 3D we will use a dimensionality reduction technique called [T-SNE](https://distill.pub/2016/misread-tsne/).

Lets get the article embeddings first.

In [0]:
import nltk
from nltk.corpus import reuters

nltk.download("reuters")
nltk.download("punkt")

max_samples = 256
categories = ['wheat', 'tea', 'strategic-metal', 
              'housing', 'money-supply', 'fuel']

S, X, Y = [], [], []

for category in categories:
  print(category)
  
  sents = reuters.sents(categories=category)
  sents = [' '.join(sent) for sent in sents][:max_samples]
  X.append(bert_vectorizer(sents, verbose=True))
  Y += [category] * len(sents)
  S += sents
  
X = np.vstack(X) 
X.shape

The interactive visualization of generated embeddings is available on the [Embedding Projector](https://projector.tensorflow.org/?config=https://gist.githubusercontent.com/gaphex/7262af1e151957b1e7c638f4922dfe57/raw/3b946229fc58cbefbca2a642502cf51d4f8e81c5/reuters_proj_config.json). 

From the link you can run T-SNE yourself, or load a checkpoint using the bookmark in lower-right corner (loading works only on Chrome).

To reproduce the input files used for this visualization, run the code below. Then, download the files to your machine and upload to Projector

(you can dowload files from the menu opened by the ">" button in the upper-left)

In [0]:
with open("embeddings.tsv", "w") as fo:
  for x in X.astype('float16'):
    line = "\t".join([str(v) for v in x])
    fo.write(line + "\n")

with open("metadata.tsv", "w") as fo:
  fo.write("Label\tSentence\n")
  for y, s in zip(Y, S):
    fo.write("{}\t{}\n".format(y, s))

Here's what I captured using the Projector.

In [0]:
from IPython.display import HTML

HTML("""
<video width="900" height="632" controls>
  <source src="https://storage.googleapis.com/bert_resourses/reuters_tsne_hd.mp4" type="video/mp4">
</video>
""")

## Step 5: building a nearest neighbour search engine

Now, let's say we have a knowledge base of 50k test samples, and we need to answer queries based on this data, fast. 

How do we retrieve the sample, most similar to a query, from a text database? The answer is nearest neighbour search.

Formally, the search problem we will be solving is defined as follows:

*given a set of points **S** in vector space **M**, and a query point **Q** ∈ **M**, find the closest point in **S** to **Q**.* 

There are multiple ways to define 'closest' in vector space, but we will use Euclidean distance.

So, to build an Information Retrieval system, we will follow these steps:

1.   Vectorize all samples from the knowledge base - that gives **S**
2.   Vectorize the query - that gives **Q**
3.   Compute euclidean distances **D** between **Q** and **S**
4.   Sort **D** in ascending order , providing indices of the most similar samples
5.   Retrieve labels for said samples from the knowledge base

To make the simple matter of implementing this a bit more exciting, we will do it in pure TensorFlow.

First we define the placeholders for Q and S

In [0]:
graph = tf.Graph()
sess = tf.InteractiveSession(graph=graph)

dim = X.shape[1]

Q = tf.placeholder("float", [dim])
S = tf.placeholder("float", [None, dim])

Define distance computation

In [0]:
squared_distance = tf.reduce_sum(tf.pow(Q - S, 2), reduction_indices=1)
distance = tf.sqrt(squared_distance)

Finally, get the most similar indices

In [0]:
top_k = 10

top_neg_dists, top_indices = tf.math.top_k(tf.negative(distance), k=top_k)
top_dists = tf.negative(top_neg_dists)

Sanity check

In [0]:
from sklearn.metrics.pairwise import euclidean_distances

In [0]:
top_indices.eval({Q:X[0], S:X})

In [0]:
np.argsort(euclidean_distances(X[:1], X)[0])[:10]

## Step 6: accelerating search

Now that we have a basic retrieval engine set up, the question is: can we make it run faster? 
With a tiny bit of math, we can.



For a pair of vectors p and q the euclidean distance is defined as follows:

$$ d(\mathbf{p}, \mathbf{q}) = d(\mathbf{q}, \mathbf{p}) = \sqrt{(p_{1}-q_{1})^{2}+(p_{2}-q_{2})^{2}+ ... + (p_{n}-q_{n})^{2}} = \sqrt{\sum_{i=1}^{n}(p_{i}-q_{i})^{2}}$$

Which is exactly how we did compute it in step 4.

However, since p and q are vectors, we can expand and rewrite this:

$$ d(\mathbf{p}, \mathbf{q})^{2} = \langle \mathbf{p} - \mathbf{q}, \mathbf{p} - \mathbf{q} \rangle  =\langle \mathbf{p} \mathbf{p} \rangle - 2 \langle \mathbf{p} \mathbf{q} \rangle + \langle \mathbf{q} \mathbf{q} \rangle $$

where ⟨…⟩ denotes inner product.

In TensorFlow this can be written as follows:

In [0]:
Q = tf.placeholder("float", [dim])
S = tf.placeholder("float", [None, dim])

Qr = tf.reshape(Q, (1, -1))

PP = tf.keras.backend.batch_dot(S, S, axes=1)
QQ = tf.matmul(Qr, tf.transpose(Qr))
PQ = tf.matmul(S, tf.transpose(Qr))

distance = PP - 2 * PQ + QQ
distance = tf.sqrt(tf.reshape(distance, (-1,)))

top_neg_dists, top_indices = tf.math.top_k(tf.negative(distance), k=top_k)

In [0]:
top_indices.eval({Q:X[0], S:X})

By the way, in the formula above PP and QQ are actually squared [L2 norms](https://en.wikipedia.org/wiki/Norm_(mathematics)#Euclidean_norm) of the respective vectors. If both vectors are L2 normalized, then PP = QQ = 1. 

That gives an interesting relation between inner product and euclidean distance for L2 normalized data:

$$d(\mathbf{p}, \mathbf{q})^{2} = 2 * (1 - \langle \mathbf{p} \mathbf{q} \rangle )$$

However, doing L2 normalization discards the information about vector magnetude, which is undesirable in many cases. 

Instead, we may notice that as long as the knowledge base does not change, PP, it's vector norm, also remains constant. So, instead of recomputing it every time, we can just do it once and then use the precomputed result, further accelerating the distance computation.

In [0]:
Q = tf.placeholder("float", [dim])
S = tf.placeholder("float", [None, dim])
S_norm = tf.placeholder("float", [None, 1])

Qr = tf.reshape(Q, (1, -1))

PP = S_norm
QQ = tf.matmul(Qr, tf.transpose(Qr))
PQ = tf.matmul(S, tf.transpose(Qr))

distance = PP - 2 * PQ + QQ
distance = tf.sqrt(tf.reshape(distance, (-1,)))

top_neg_dists, top_indices = tf.math.top_k(tf.negative(distance), k=top_k)

In [0]:
X_norm = np.sum(X**2, axis=1).reshape((-1,1))

In [0]:
top_indices.eval({Q:X[0], S:X, S_norm: X_norm})

Now, let us put it all together:

In [0]:
class L2Retriever:
    def __init__(self, dim, top_k=3, use_norm=False, use_gpu=True):
        self.dim = dim
        self.top_k = top_k
        self.use_norm = use_norm
        config = tf.ConfigProto(
            device_count={'GPU': (1 if use_gpu else 0)}
        )
        self.session = tf.Session(config=config)
        
        self.norm = None
        self.query = tf.placeholder("float", [self.dim])
        self.kbase = tf.placeholder("float", [None, self.dim])
        
        self.build_graph()

    def build_graph(self):
      
        if self.use_norm:
            self.norm = tf.placeholder("float", [None, 1])

        distance = dot_l2_distances(self.kbase, self.query, self.norm)
        top_neg_dists, top_indices = tf.math.top_k(tf.negative(distance), k=self.top_k)
        top_dists = tf.negative(top_neg_dists)

        self.top_distances = top_dists
        self.top_indices = top_indices

    def predict(self, kbase, query, norm=None):
        query = np.squeeze(query)
        feed_dict = {self.query: query, self.kbase: kbase}
        if self.use_norm:
          feed_dict[self.norm] = norm
        
        I, D = self.session.run([self.top_indices, self.top_distances],
                                feed_dict=feed_dict)
        
        return I, D
      
def dot_l2_distances(kbase, query, norm=None):
    query = tf.reshape(query, (1, -1))
    
    if norm is None:
      XX = tf.keras.backend.batch_dot(kbase, kbase, axes=1)
    else:
      XX = norm
    YY = tf.matmul(query, tf.transpose(query))
    XY = tf.matmul(kbase, tf.transpose(query))
    
    distance = XX - 2 * XY + YY
    distance = tf.sqrt(tf.reshape(distance, (-1,)))
    
    return distance

## Example: movie recommendation system

For this example we will use a dataset of movie summaries from IMDB. 

We will vectorize the reviews and use the search engine to find nearest neighbours in movie space! 

That will allow us to build a simple movie recommendation system, which will suggest similar movies based on plot features.

First let's download and prepare the dataset.

In [0]:
import pandas as pd
import json

# download and build the dataset 
!wget http://www.cs.cmu.edu/~ark/personas/data/MovieSummaries.tar.gz
!tar -xvzf MovieSummaries.tar.gz

plots_df = pd.read_csv('MovieSummaries/plot_summaries.txt', sep='\t', header=None)
meta_df = pd.read_csv('MovieSummaries/movie.metadata.tsv', sep='\t', header=None)

plot = {}
metadata = {}
movie_data = {}

for movie_id, movie_plot in plots_df.values:
  plot[movie_id] = movie_plot 

for movie_id, movie_name, movie_genre in meta_df[[0,2,8]].values:
  genre = list(json.loads(movie_genre).values())
  if len(genre):
    metadata[movie_id] = {"name": movie_name,
                          "genre": genre}
    
for movie_id in set(plot.keys())&set(metadata.keys()):
  movie_data[metadata[movie_id]['name']] = {"genre": metadata[movie_id]['genre'],
                                            "plot": plot[movie_id]}

X, Y, names = [], [], []

for movie_name, movie_meta in movie_data.items():
  X.append(movie_meta['plot'])
  Y.append(movie_meta['genre'])
  names.append(movie_name)

Vectorize movie plots with BERT NLU. This will take a while.

In [0]:
X_vect = bert_vectorizer(X, verbose=True)

In [0]:
X_square_norm = np.sum(X_vect**2, axis=1).reshape((-1,1))

On CPU, using precomputed norm yields a ~4X improvement over the non-optimized implementation.

In [0]:
test_id = 10060 # The Matrix

retriever = L2Retriever(768, use_norm=True, top_k=10, use_gpu=False)
%timeit retriever.predict(X_vect, X_vect[test_id], X_square_norm)[0][1:]

In [0]:
retriever = L2Retriever(768, use_norm=False, top_k=10, use_gpu=False)
%timeit retriever.predict(X_vect, X_vect[test_id])[0][1:]

In [0]:
%timeit euclidean_distances(X_vect, X_vect[test_id:test_id+1])

In [0]:
def buildMovieRecommender(movie_names, vectorized_plots, top_k=10):
  retriever = L2Retriever(vectorized_plots.shape[1], use_norm=True, top_k=top_k, use_gpu=False)
  vectorized_norm = np.sum(vectorized_plots**2, axis=1).reshape((-1,1))
  
  def recommend(query):
    try:
      idx = retriever.predict(vectorized_plots, 
                              vectorized_plots[movie_names.index(query)], 
                              vectorized_norm)[0][1:]
      for i in idx:
        print(names[i])
    except ValueError:
      print("{} not found in movie db. Suggestions:")
      for i, name in enumerate(movie_names):
        if query.lower() in name.lower():
          print(i, name)
          
  return recommend

In [0]:
recommend = buildMovieRecommender(names, X_vect)

In [0]:
recommend("The Matrix")

In [0]:
recommend("Terminator 2: Judgment Day")

In [0]:
recommend("Star Wars Episode V: The Empire Strikes Back")

In [0]:
recommend("Seven Samurai")

Even without supervision, the model performs adequately on classification and retrieval tasks. While the model performance can be improved with supervised data, the described approach to text feature extraction provides a solid baseline for downstream NLP solutions. 

This conludes the guide to building a search engine with BERT and Tensorflow.
